## Preprocessing and feature engineering

In the previous module, you performed some exploratory data analysis and preprocessing in the notebook using Python scripts. In this module, you will use [Amazon SageMaker Processing](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html), which allows you to run preprocessing on fully managed infrastructure, to create and run a processing job. You will leverage the same feature transformer model you created in the previous module using SKLearn.

In [ ]:
import sagemaker, pandas, numpy
import boto3

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
s3_bucket_name = sagemaker_session.default_bucket()
s3_key_prefix = 'end-to-end-ml'

print(region)
print(role)
print(s3_bucket_name)

You will use the same dataset as the previous module. The "AI4I 2020 Predictive Maintenance Dataset" is available on the UCI Machine Learning Repository.

Download the dataset from https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset and upload it to the default S3 bucket for your SageMaker session.

In [ ]:
import urllib

dataset_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv"
raw_data_file_name = "predictive_maintenance_raw_data_header.csv"
urllib.request.urlretrieve(dataset_url, raw_data_file_name)

In [ ]:
raw_data_key = f'{s3_key_prefix}/data/raw'
sagemaker_session.upload_data(raw_data_file_name, s3_bucket_name, key_prefix=raw_data_key)

<h2>Data Exploration</h2>

You have already performed data exploration in the previous module, so you are not going to repeat the same process in this module. However, you can load the downloaded file into a dataframe and explore the data further if needed.

In [ ]:
import pandas as pd

df = pd.read_csv(raw_data_file_name)
print('The shape of the dataset is:', df.shape)

## Create an experiment

Again, you will leverage [Amazon SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) to track your experiments. Remember that each experiment is a collection of runs. Each run is a collection of inputs, parameters, configurations, and results of an iteration in the ML lifecycle.

You create an **experiment** to track a series of iterations including processing and training jobs, and create a new **run** each time you run a processing or a training job.

Choose a name of the experiment.

In [ ]:
from sagemaker.experiments.run import Run

import time
experiment_name = f"ml-end-to-end-{time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())}"
print(f"Experimentation name: {experiment_name}")

From now on, you will use the experiment name for the preprocessing and training jobs, so persist it using the store magic.

In [ ]:
%store experiment_name

<h2>Preprocessing and Feature Engineering with Amazon SageMaker Processing</h2>

The preprocessing and feature engineering code is implemented in the `source_dir/preprocessor.py` file. The preprocessing logic in this file is similar to the previous module. However, this time, you will use SageMaker Processing to run the preprocessing logic. 

The script splits the dataset into train, validation, and test sets and transforms the data. It then saves the data sets to specific output paths. After SageMaker Processing runs the script, it automatically copies the contents from the local directories in the processing container to Amazon S3. The featurizer model is needed later for transofrming the features during inference, so the script also saves the featurizer model for later use.

In [ ]:
!pygmentize source_dir/preprocessor.py

Configuring an Amazon SageMaker Processing job through the SM Python SDK requires to create a `Processor` object. In this case you will be using the default SKLearn container for processing, so you will use `SKLearnProcessor`. The object takes a number of parameters including the number of type of instances to use.

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(role=role,
                                     base_job_name='sm-feature-engineering',
                                     instance_type='ml.m5.large',
                                     instance_count=1,
                                     framework_version='0.20.0')

Invoke the `run` method of the `SKLearnProcessor` object to kick-off the job, specifying the script to execute, the arguments to pass to the script, and the configuration of inputs and outputs.

In [ ]:
run_name=f'feature-engineering-{time.strftime("%H-%M-%S", time.localtime())}'
run_display_name=run_name

raw_data_path = f's3://{s3_bucket_name}/{s3_key_prefix}/data/raw/'
train_data_path = f's3://{s3_bucket_name}/{s3_key_prefix}/data/preprocessed/train/'
val_data_path = f's3://{s3_bucket_name}/{s3_key_prefix}/data/preprocessed/val/'
test_data_path = f's3://{s3_bucket_name}/{s3_key_prefix}/data/preprocessed/test/'
model_path = f's3://{s3_bucket_name}/{s3_key_prefix}/output/sklearn/'

with Run(
    experiment_name=experiment_name,
    run_name=run_name,
    run_display_name=run_display_name,
    sagemaker_session=sagemaker_session,
) as run:

    sklearn_processor.run(code='source_dir/preprocessor.py',
                          inputs=[ProcessingInput(input_name='raw_data', source=raw_data_path, destination='/opt/ml/processing/input')],
                          outputs=[ProcessingOutput(output_name='train_data', source='/opt/ml/processing/train', destination=train_data_path),
                                   ProcessingOutput(output_name='val_data', source='/opt/ml/processing/val', destination=val_data_path),
                                   ProcessingOutput(output_name='test_data', source='/opt/ml/processing/test', destination=test_data_path),
                                   ProcessingOutput(output_name='model', source='/opt/ml/processing/model', destination=model_path)],
                          arguments=['--train-test-split-ratio', '0.2'])

While the job is running, you can review its configurations, logs, and metrics under the Processing tab in the SageMaker Console (not in SageMaker Studio).

Once the job is completed, you can analyze the preprocessed dataset by downloading one of the feature files from S3 and loading it into a dataframe.

In [ ]:
validation_features_key_prefix = f'{s3_key_prefix}/data/preprocessed/val/val_features.csv'

sagemaker_session.download_data('./', s3_bucket_name, validation_features_key_prefix)

In [ ]:
import pandas as pd
df = pd.read_csv('val_features.csv')

df.head(10)

You can see that the categorical variables have been one-hot encoded. Feel free to check that we do not have NaN values anymore, as expected.

### Experiment analytics

As before, you can visualize experiment analytics using Amazon SageMaker Studio Experiments ([learn more](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments-view-compare.html)) or use the SageMaker SDK to display the experiment analytics, as follows:

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

analytics = ExperimentAnalytics(experiment_name=experiment_name)
analytics.dataframe()

After the preprocessing and feature engineering are completed, you can move to the next notebook in the **04_train_model** folder to start model training.